In [ ]:
from shapely.geometry import Polygon
from ultralytics import YOLO
import cv2
import os
from PIL import Image

In [ ]:
# bibliotecas
from dataclasses import dataclass
from pycocotools.coco import COCO
from ultralytics import YOLO
from pathlib import Path
from shapely.geometry import Polygon
import pandas as pd

def coco_to_xsys(segmentation):
    return [(segmentation[i], segmentation[i + 1]) for i in range(0, len(segmentation), 2)]

# import do script usado para detecção de veículos
from parking_detector import ParkingSpace, Vehicle

# modelo de detecção
model = YOLO("yolov8x-seg.pt")

# categoria e id dos espaços de estacionamento
parking_names = list("abcdef")
test_selection = [
    [(381, 193), (377, 319), (502, 339), (486, 179), (396, 174)],
    [(309, 142), (277, 245), (386, 300), (395, 152)],
    [(248, 91), (203, 193), (282, 232), (329, 114)],
    [(193, 54), (138, 155), (202, 185), (258, 75)],
    [(149, 30), (78, 124), (136, 142), (205, 45)],
    [(113, 4), (48, 70), (87, 97), (166, 21)],
]

# lista de objetos ParkingSpace
parking_spaces = []

# criação dos objetos ParkingSpace
for space_id, space_name in enumerate(parking_names):
    parking_spaces.append(ParkingSpace(space_id, test_selection[space_id]))

path = Path("/home/ren/Downloads/carscoco/valid")
annotations_coco = COCO(path / "_annotations.coco.json")
ann_data = annotations_coco.imgToAnns

i = 0
correct = 0
wrong = 0

data = []
for image in annotations_coco.imgs.values():
    # valores reais
    file_name = path / image["file_name"]
    id = image["id"]
    anns = ann_data[id]
    cats_actual = [ann["category_id"]-1 for ann in anns if ann['category_id']-1 != 6]
    true_segs = {ann['category_id']-1: ann['segmentation'][0] for ann in anns if ann['category_id']-1 !=6}

    # valores preditos pelo modelo
    results = model(file_name, classes=[1, 2, 7])
    car_seg_preds = results[0].masks.xy
    vehicles = [
        Vehicle(None, None, None, polygon=Polygon(mask)) for mask in car_seg_preds
    ]
    ordered_vehicles = [None]*6
    for vehicle in vehicles:
        for parking_space in parking_spaces:
            if not parking_space.intersects(vehicle, threshold=0.3):
                continue

            ordered_vehicles[parking_space.id] = vehicle
            if vehicle.cls_id is not None:
                vehicle.cls_id = max(parking_space.id, vehicle.cls_id)
                continue

            vehicle.cls_id = parking_space.id
    cats_predicted = [vehicle.cls_id for vehicle in ordered_vehicles if vehicle is not None]

    for cat in range(len(parking_names)):
        iou = None
        if(cat not in cats_actual and cat not in cats_predicted):
            continue
        if cat in cats_actual and cat in cats_predicted:
            true_seg = coco_to_xsys(true_segs[cat])
            pol_actual = Polygon(true_seg)
            pol_predicted = ordered_vehicles[cat].polygon
            iou = pol_predicted.intersection(pol_actual).area/pol_predicted.union(pol_actual).area
            wrong = wrong + 1 if iou < .3 else wrong
            correct = correct + 1 if iou > .3 else correct
        else:
            wrong+=1

        data.append({
                        'ImageFileName': file_name,
                        'ImageID': id,
                        'catID': cat,
                        'catName': parking_names[cat],
                        'predicted': cat in cats_predicted,
                        'actual': cat in cats_actual,
                        'IOU': iou
                    })
    i+=1
    print(f"step {i} wrong {wrong} | total {correct+wrong} | acc {correct/(correct+wrong)}")
df = pd.DataFrame(data)

In [ ]:
grouped = df.groupby('catID')

accuracy_per_cat = grouped.apply(lambda group: (group['predicted'] == group['actual']).mean())

In [ ]:
grouped = df.groupby('catName')

def calculate_metrics(group):
    total_instances = len(group)
    correct_guesses = (group['predicted'] == group['actual']).sum()
    wrong_guesses = total_instances - correct_guesses
    accuracy = correct_guesses / total_instances if total_instances > 0 else 0.0
    return pd.Series({
        'Total': total_instances,
        'Correto': correct_guesses,
        'Errado': wrong_guesses,
        'Acurácia': accuracy,
        'Número de instâncias': total_instances,
    })

metrics_per_cat = grouped.apply(calculate_metrics)

In [ ]:
metrics_per_cat.rename_axis("Vaga")

In [ ]:
metrics_per_cat.Correto.sum()/metrics_per_cat.Total.sum()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 6)) 
ax.axis('tight')
ax.axis('off')
ax.table(cellText=metrics_per_cat.values, colLabels=metrics_per_cat.columns, cellLoc = 'center', loc='center')
plt.savefig('metrics_table.png', bbox_inches='tight', pad_inches=0.1)
plt.show()